In [2]:
from core.utils import Tibanna
from core import ff_utils
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

asets = [
            'dcic:Selvaraj_gm12878_hic',
            'dekker-lab:ExperimentSet_U54_U54-ESC4DN-FA-DpnII-2017524',
            'dekker-lab:ExperimentSet_U54_HFFc6-FA-DpnII'
            ]


env = 'fourfront-webprod'
tibanna = Tibanna(env=env)

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)


def form_hyp(id):
    hyp = '=HYPERLINK("https://data.4dnucleome.org/{0}","{0}")'.format(id)
    return hyp



Fs = []
my_rep_set = asets[2]

rep_resp = ff_utils.get_metadata(my_rep_set, connection=ff)
Fcol = form_hyp(rep_resp['accession'])
for exp in rep_resp['experiments_in_set']:
    exp_resp = ff_utils.get_metadata(exp, connection=ff)
    Scol = form_hyp(exp_resp['accession'])
    for fil in exp_resp['files']:
        fil_resp = ff_utils.get_metadata(fil, connection=ff)
        Tcol = form_hyp(fil_resp['accession'])
        FOcol = fil_resp['uuid']
        if fil_resp['paired_end'] == '1':
            print '\t'.join([Fcol,Scol,Tcol,FOcol,"1"])
            Fs.append(fil_resp['file_size'])
            rel_fil = fil_resp['related_files'][0]['file']
            fil2_resp = ff_utils.get_metadata(rel_fil, connection=ff)
            Tcol = form_hyp(fil2_resp['accession'])
            FOcol = fil2_resp['uuid']
            if fil2_resp['paired_end'] == '2':
                print '\t'.join([Fcol,Scol,Tcol,FOcol,"2"])
                Fs.append(fil2_resp['file_size'])

                
for i in Fs:
    print round(i/(1024*1024*1024.0),2)
 

#     print my_rep_set
#     my_aliases = []
#     my_files = []
#     for exp in rep_resp:
#         exp_resp = ff_utils.get_metadata(exp, connection=ff)
#         exp_files = exp_resp['files']
#         al = exp_resp['aliases'][0].split(':')[1].upper()
         
#         used_file = False
#         for fastq_file in exp_files:
#             file_resp = ff_utils.get_metadata(fastq_file, connection=ff)  
#             if file_resp.get('instrument', 'Illumina my seq') != 'Illumina MiSeq':
#                 used_file = True
#                 f_al = file_resp['aliases'][0]
#                 if f_al.startswith('dcic:H'):
#                     my_files.append(f_al.split('_')[1].upper())
#                 else:
#                     my_files.append(f_al.split('_')[2].upper())
       
    
#         if used_file:
#             my_aliases.append(al)
#     print ', '.join(my_aliases)
#     print len(list(sorted(set(my_files))))
#     print ', '.join(list(sorted(set(my_files))))



=HYPERLINK("https://data.4dnucleome.org/4DNES2R6PUEK","4DNES2R6PUEK")	=HYPERLINK("https://data.4dnucleome.org/4DNEX7POCO84","4DNEX7POCO84")	=HYPERLINK("https://data.4dnucleome.org/4DNFIWDY2U4S","4DNFIWDY2U4S")	01795b9f-b843-4ac6-8391-7ef2b0497782	1
=HYPERLINK("https://data.4dnucleome.org/4DNES2R6PUEK","4DNES2R6PUEK")	=HYPERLINK("https://data.4dnucleome.org/4DNEX7POCO84","4DNEX7POCO84")	=HYPERLINK("https://data.4dnucleome.org/4DNFIS8I1Y5L","4DNFIS8I1Y5L")	d9f827b1-d3c6-4139-8287-e8e0fedccc5e	2
=HYPERLINK("https://data.4dnucleome.org/4DNES2R6PUEK","4DNES2R6PUEK")	=HYPERLINK("https://data.4dnucleome.org/4DNEX7POCO84","4DNEX7POCO84")	=HYPERLINK("https://data.4dnucleome.org/4DNFI65A88XH","4DNFI65A88XH")	88d1054a-769b-4425-bd26-9b519e4aa39a	1
=HYPERLINK("https://data.4dnucleome.org/4DNES2R6PUEK","4DNES2R6PUEK")	=HYPERLINK("https://data.4dnucleome.org/4DNEX7POCO84","4DNEX7POCO84")	=HYPERLINK("https://data.4dnucleome.org/4DNFIY67X18K","4DNFIY67X18K")	b4fdd2bb-bc42-4a98-b86d-352baf2c053e	2
=HYP

=HYPERLINK("https://data.4dnucleome.org/4DNES2R6PUEK","4DNES2R6PUEK")	=HYPERLINK("https://data.4dnucleome.org/4DNEXRAEERUF","4DNEXRAEERUF")	=HYPERLINK("https://data.4dnucleome.org/4DNFIWZUG4AN","4DNFIWZUG4AN")	3aaf1c99-4599-4ecb-8d66-f42935abd64f	1
=HYPERLINK("https://data.4dnucleome.org/4DNES2R6PUEK","4DNES2R6PUEK")	=HYPERLINK("https://data.4dnucleome.org/4DNEXRAEERUF","4DNEXRAEERUF")	=HYPERLINK("https://data.4dnucleome.org/4DNFI65C492U","4DNFI65C492U")	d6a72d2f-d361-43e6-8ea5-2fa26ce8295e	2
10.23
10.64
10.03
10.64
10.07
10.76
10.33
10.81
10.32
10.78
10.08
10.74
9.9
10.59
10.42
10.85
10.43
11.43
10.6
11.01
10.44
10.9
10.51
10.96
10.13
10.78
10.12
10.73
10.54
10.97
10.16
10.76
10.03
10.77
10.7
11.76
